## ML Lab 10 - Building a CNN model in Keras (Happy House) 

**Objectives**:  Build a deep learning algorithm using Keras, a high-level neural networks programming framework, written in Python and capable of running on top of TensorFlow.  

Keras was developed to enable deep learning engineers to build and experiment with different models very quickly. Just as TensorFlow is a higher-level framework than Python, Keras is an even higher-level framework and provides additional abstractions. 

**Note**: We import a lot of functions from Keras. You can use them easily just by calling them directly in the notebook. Ex: `X = Input(...)` or `X = ZeroPadding2D(...)`.


In [14]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)

# Import relevant libraries
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline


##  Problem:  Happy House Challenge

You decided to spend your vacation with friends in a very convenient holiday house with many things to do nearby. But the most important benefit is that everybody has commited to be happy when they are in the house. So anyone wanting to enter the house must prove their current state of happiness.

As a deep learning expert, to make sure the "Happy" rule is strictly applied, you are going to build an algorithm that uses pictures from the front door camera to check if the person is happy or not. The door should open only if the person is happy. 

You have gathered pictures of your friends and yourself, taken by the front-door camera. The dataset is labbeled. 

<img src="images/house-members.png" style="width:500px;height:250px;">

**Details of the "Happy" dataset**:
- Images are of shape (64,64,3)
- Training: 600 pictures
- Test: 150 pictures

Run the following code to load and normalize the dataset and learn about its shapes.

In [18]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshape
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 600
number of test examples = 150
X_train shape: (600, 64, 64, 3)
Y_train shape: (600, 1)
X_test shape: (150, 64, 64, 3)
Y_test shape: (150, 1)


## Building a model in Keras

Here is an example of a model in Keras:

```python
def model(input_shape):
    # Define the input placeholder as a tensor with shape input_shape. 
    #Think of this as an input image.
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    #Conv2D(filters, kernel_size, .... data_format='channels_last',...)
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    
    #Normalize the activations of the previous layer at each batch, i.e. applies a transformation     that maintains the mean activation close to 0 and activation standard deviation close to 1.
    # If data_format="channels_last", the axis to be normalized is axis=3. 
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL: helps to lower the dimension of X in height and width.
    X = MaxPooling2D((2, 2), name='max_pool')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Creates the Keras model instance (object), you'll use to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='HappyModel')
    
    return model
```

Note that Keras uses a different convention with variable names than we ussually do with neural network layers. In particular, rather than creating and assigning a new variable on each step of forward propagation such as `X`, `Z1`, `A1`, `Z2`, `A2`, etc. for the computations for the different layers, in Keras code each line above just reassigns `X` to a new value using `X = ...`. In other words, during each step of forward propagation, we are just writing the latest value in the computation into the same variable `X`. The only exception was `X_input`, which we kept separate and did not overwrite, since we needed it at the end to create the Keras model instance (`model = Model(inputs = X_input, ...)` above). 

**Exercise**: Implement a `HappyModel()`. We suggest that you start by implementing the architecture proposed above as your initial model and run it. After that, come back and vary the network architecture (e.g. add new blocks CONV -> BN -> RELU -> MAXP) or vary the hyperparameters (e.g Conv2D(32, (3, 3),....) ).  

Be careful with the data's shapes. 

In [3]:
# HappyModel

def HappyModel(input_shape):
    """
    Implementation of the HappyModel.
    
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """
    # Use the suggested model in the text above to get started, and run through the whole
    # exercise once. Then come back and add more BLOCKS. 
    X_input = Input(input_shape)
    
    X = ZeroPadding2D((2, 2))(X_input)
    # BLOCK 1: CONV -> BN -> RELU -> MAXP
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)
    X = AveragePooling2D((2, 2), name='max_pool')(X)

    # BLOCK 2: CONV -> BN -> RELU -> MAXP

        
    # BLOCK N: CONV -> BN -> RELU -> MAXP
       
        
    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Create model.
    model = Model(inputs = X_input, outputs = X, name='HappyModel')

    
    return model

You have now built a function to describe your model. 

To train and test this model, there are four steps in Keras (Create->Compile->Fit/Train->Evaluate/Test:

1. Create the model by calling the function above
2. Compile the model by calling `model.compile(optimizer = "...", loss = "...", metrics = ["accuracy"])`
3. Train the model on train data by calling `model.fit(X_train, Y_train, epochs = ..., batch_size = ...)`
4. Test the model on test data by calling `model.evaluate(x = ..., y = ...)`

If you want to know more about `model.compile()`, `model.fit()`, `model.evaluate()` and their arguments, refer to the official [Keras documentation](https://keras.io/models/model/).


In [19]:
# 1. Create the model by calling the function above
happyModel = HappyModel(X_train.shape[1:])


In [21]:
# 2.Compile the model to configure the learning process. Choose the arguments of compile() wisely.
#Hint: the Happy Challenge is a binary classification problem.
happyModel.compile(optimizer="adam", loss="mean_squared_error", metrics=["accuracy"])


In [22]:
# 3.Train the model on train data. Choose the number of epochs and the batch size.
happyModel.fit(X_train,Y_train, epochs = 1000, batch_size=2)


Epoch 1/1000


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv0_3/convolution (defined at /home/gabriel/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_3122]

Function call stack:
keras_scratch_graph


Note that if you run `fit()` again, the `model` will continue to train with the parameters it has already learnt instead of reinitializing them.


In [ ]:
#Test the model on test data
preds = happyModel.evaluate()

print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

If `happyModel()` function worked, you should have observed much better than random-guessing (50%) accuracy on the train and test sets. You have to get at least 75% accuracy. 
Even if you have achieved 75% accuracy, please feel free to keep playing with your model to try to get even better results. To give you a point of comparison, our model gets around 95% test accuracy in 40 epochs (and 99% train accuracy) with a mini batch size of 16 and "adam" optimizer. Depending on your computer it may take some time to finish the complete set of training epochs. 

- Change your optimizer. We find Adam works well. 
- If the model is struggling to run and you get memory issues, lower your batch_size. 
- Run on more epochs, until you see the train accuracy plateauing. 


In [ ]:
#Prints the details of your layers in a table with the sizes of its inputs/outputs
happyModel.summary()

## Test with your own image (Optional)

Take a picture of your face and see if you could enter the Happy House. To do that, add your image to this Jupyter Notebook's directory, in the "images" folder and run the code below and check if the algorithm is right (0 is unhappy, 1 is happy)!
    
NOTE: All pictures (the training and test sub-sets) for this assignment were taken against the same background (since a front door camera is always mounted in the same position). This makes the problem easier, but a model trained on this data may or may not work on your own data. But feel free to give it a try! 

In [ ]:
#your own image "my_image.jpg". 
img_path = 'images/my_image.jpg'

img = image.load_img(img_path, target_size=(64, 64))
imshow(img)

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

print(happyModel.predict(x))